In [2]:
import requests
import pandas as pd
import ast
import sqlalchemy
from sqlalchemy import Column, String, Float, Integer
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import func
from config import nps_key

In [3]:
import urllib.request, json

base_url = "https://developer.nps.gov/api/v1/parkinglots?"
api_key = nps_key


In [4]:
indices = [0,50,100,150,200,250,300,350,400,450]

parkinglotdata = []

In [5]:
for index in indices: 
    url = (f"{base_url}&start={index}&api_key={api_key}")
    response = requests.get(url).json()

    data = response['data']

    for i in range(len(data)):

        entry = data[i]

        parkinglotdata.append(entry)   

In [6]:
parkinglot_df = pd.DataFrame(parkinglotdata)
parkinglot_df.head()

,id,name,altName,description,relatedParks,latitude,longitude,geometryPoiId,managedByOrganization,timeZone,webcamUrl,contacts,fees,operatingHours,images,accessibility,liveStatus
0,4FB3D0C3-18BB-40BB-8AB5-FC78E543DD06,3rd Street Parking,3rd Street Parking,Street parking starting at the corner of 3rd s...,"[{'states': 'AR,OK', 'parkCode': 'fosm', 'desi...",35.387385,-94.429605,,Fort Smith National Historic Site,CT,,"{'phoneNumbers': [], 'emailAddresses': []}",[],[],[],"{'isLotAccessibleToDisabled': True, 'totalSpac...","{'isActive': True, 'occupancy': '', 'estimated..."
1,C1E8B30D-D0E2-49AF-88F4-B536B9B65371,49 Palms Oasis,,The small parking area for the 49 Palms Oasis ...,"[{'states': 'CA', 'parkCode': 'jotr', 'designa...",34.119568,-116.112168,,The National Park Service,PT,,"{'phoneNumbers': [], 'emailAddresses': []}","[{'cost': '0.00', 'description': 'The 49 Palms...",[{'exceptions': [{'exceptionHours': {'wednesda...,[],"{'isLotAccessibleToDisabled': False, 'totalSpa...","{'isActive': True, 'occupancy': 'Closed', 'est..."
2,B5112F5C-F376-46F0-9EFC-E76388029363,ADA Parking,Rear Parking,ADA parking spaces available behind the Visito...,"[{'states': 'KS', 'parkCode': 'brvb', 'designa...",39.037983,-95.676738,,Brown v. Board of Education NHS,CST,,"{'phoneNumbers': [], 'emailAddresses': []}","[{'cost': '0.00', 'description': 'ADA Spaces b...","[{'exceptions': [], 'description': '', 'standa...",[],"{'isLotAccessibleToDisabled': True, 'totalSpac...","{'isActive': True, 'occupancy': '', 'estimated..."
3,2F2D55B3-14C5-4374-98C9-78088FDE6640,Abbotts Bridge Parking Lot,,The parking lot is centrally located near the ...,"[{'states': 'GA', 'parkCode': 'chat', 'designa...",34.023899,-84.171415,,NPS,ET,,"{'phoneNumbers': [], 'emailAddresses': []}","[{'cost': '5.00', 'description': 'The CRNRA Da...","[{'exceptions': [], 'description': '', 'standa...","[{'credit': 'NPS', 'title': 'Abbotts Bridge En...","{'isLotAccessibleToDisabled': False, 'totalSpa...","{'isActive': True, 'occupancy': '', 'estimated..."
4,2931685F-746F-4A72-ABD6-7D6CDA191E3D,Abbotts Lagoon Trailhead Parking Lot,Abbotts Lagoon Parking Lot,The Abbotts Lagoon Trailhead Parking Lot is a ...,"[{'states': 'CA', 'parkCode': 'pore', 'designa...",38.123453,-122.935334,,National Park Service,PT,,{'phoneNumbers': [{'phoneNumber': '415-464-510...,"[{'cost': '0.00', 'description': 'No parking o...","[{'exceptions': [], 'description': '', 'standa...","[{'credit': 'NPS Photo', 'title': 'Abbotts Lag...","{'isLotAccessibleToDisabled': True, 'totalSpac...","{'isActive': False, 'occupancy': 'Light', 'est..."


In [7]:
parkinglot_df.columns

Index(['id', 'name', 'altName', 'description', 'relatedParks', 'latitude',
       'longitude', 'geometryPoiId', 'managedByOrganization', 'timeZone',
       'webcamUrl', 'contacts', 'fees', 'operatingHours', 'images',
       'accessibility', 'liveStatus'],
      dtype='object')

In [8]:
parkinglot_df = parkinglot_df.loc[:,['name', 'description', 'latitude',
       'longitude']]

parkinglot_df.head()

,name,description,latitude,longitude
0,3rd Street Parking,Street parking starting at the corner of 3rd s...,35.387385,-94.429605
1,49 Palms Oasis,The small parking area for the 49 Palms Oasis ...,34.119568,-116.112168
2,ADA Parking,ADA parking spaces available behind the Visito...,39.037983,-95.676738
3,Abbotts Bridge Parking Lot,The parking lot is centrally located near the ...,34.023899,-84.171415
4,Abbotts Lagoon Trailhead Parking Lot,The Abbotts Lagoon Trailhead Parking Lot is a ...,38.123453,-122.935334


In [9]:
engine = sqlalchemy.create_engine('sqlite:///../nps.sqlite', echo=False)


parkinglot_df.to_sql('parkinglot_df', con=engine, if_exists = "replace")

455